In [1]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

In [2]:
import argparse 

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 7

In [30]:
import matplotlib.pyplot as plt
from pathlib import Path

def plot_history(history):
    log.info("History keys: %s", history.history.keys())
    # Accuracy
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(history.history['accuracy'], label='Train')
    ax.plot(history.history['val_accuracy'], label='Test')
    ax.set_title('Model accuracy')
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('Epoch')
    ax.grid(True)
    ax.legend(['Train', 'Val'], loc='lower right')
    
    # Loss
    plt.figure(figsize=(10,5))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.grid()
    plt.show()
    
# couldn't make it work
# def build_dataset(data_folder):
#     def preprocess(line):
#         csv_records = tf.io.decode_csv(line, record_defaults=[tf.constant(0)]*4)
#         print(csv_records)
#         x = tf.stack(csv_records)
#         return x
#     dataset = tf.data.Dataset.list_files(data_folder + 'chorale_*.csv', seed=args.seed)
#     dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1))
#     dataset = dataset.map(preprocess)
# #     dataset = dataset.shuffle(10000)
#     dataset = dataset.batch(args.batch_size).prefetch(1)
#     for d in dataset:
#         print(d)
#         break
#     return dataset

def build_dataset(data_folder):
    paths = Path(data_folder).glob('**/chorale_*.csv')
    np_list = [pd.read_csv(p).to_numpy() for p in paths]
    ragged = tf.ragged.constant(np_list)
    dataset = tf.data.Dataset.from_tensor_slices(ragged)
    dataset = dataset.flat_map(lambda x: tf.data.Dataset.from_tensor_slices(x))
    print(dataset)
    for d in dataset:
        print(d)
        break
#     def preprocess(line):
#         csv_records = tf.io.decode_csv(line, record_defaults=[tf.constant(0)]*4)
#         print(csv_records)
#         x = tf.stack(csv_records)
#         return x
#     dataset = tf.data.Dataset.list_files(data_folder + 'chorale_*.csv', seed=args.seed)
#     dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1))
#     dataset = dataset.map(preprocess)
# #     dataset = dataset.shuffle(10000)
#     dataset = dataset.batch(args.batch_size).prefetch(1)

#     return dataset



def baseline_model():
    model = keras.models.Sequential()
    model.add(keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 4]))
    model.add(keras.layers.SimpleRNN(20))
    model.add(keras.layers.Dense(4))
    model.compile(loss='mse',optimizer=keras.optimizers.Nadam(learning_rate=0.01), metrics=[keras.metrics.mean_squared_error])
    return model

train_dataset = build_dataset(data_folder=args.train_folder)
# val_dataset = build_dataset(data_folder=args.val_folder)
# model = baseline_model()
# model.fit(train_dataset, epochs=1, validation_data=val_dataset)

<FlatMapDataset shapes: (None,), types: tf.int32>
tf.Tensor([66 61 57 54], shape=(4,), dtype=int32)
